In [19]:
import numpy as np
import pandas as pd

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
df = pd.read_csv('/content/drive/MyDrive/DACOS/추천시스템/숙명여대_맛집_크롤링.csv')
df.head()

,index,가게명,음식종류,태그리뷰
0,0,육쌈냉면 숙대점,냉면,"[{'""음식이 맛있어요""': 419}, {'""가성비가 좋아요""': 165}, {'""..."
1,1,장재근초밥,"초밥,롤","[{'""음식이 맛있어요""': 96}, {'""재료가 신선해요""': 56}, {'""친절..."
2,2,면식당 숙명여대점,일식당,"[{'""음식이 맛있어요""': 373}, {'""혼밥하기 좋아요""': 188}, {'""..."
3,3,포라임 숙대본점,베트남음식,"[{'""음식이 맛있어요""': 869}, {'""혼밥하기 좋아요""': 419}, {'""..."
4,4,베스트프렌드,종합분식,"[{'""음식이 맛있어요""': 91}, {'""인테리어가 멋져요""': 38}, {'""친..."


In [22]:
df['음식종류'].value_counts()

,count
음식종류,
한식,11
"카페,디저트",10
카페,7
"육류,고기요리",6
베트남음식,5
베이커리,4
일식당,4
"초밥,롤",4
"치킨,닭강정",3


In [23]:
# 카페 관련 종류와 주류 관련 종류 리스트 정의
cafe_types = ['카페,디저트', '카페', '베이커리', '와플', '브런치카페']
drink_types = ['이자카야', '요리주점', '바(BAR)', '와인', '맥주,호프']

# 조건에 따라 데이터프레임 나누기
df_cafe = df[df['음식종류'].isin(cafe_types)]
df_drink = df[df['음식종류'].isin(drink_types)]
df_meal = df[~df['음식종류'].isin(cafe_types + drink_types)]

In [24]:
import ast

# 태그리뷰 파싱 및 열 확장
def parse_tags(tag_str):
    tags = ast.literal_eval(tag_str)
    parsed_tags = {}
    for tag_dict in tags:
        for key, value in tag_dict.items():
            parsed_tags[key.strip('"')] = value
    return parsed_tags

# 태그리뷰 열을 파싱하고 새로운 열로 확장
tags_expanded = df_meal['태그리뷰'].apply(parse_tags).apply(pd.Series)
df_meal = pd.concat([df_meal.drop(columns=['태그리뷰']), tags_expanded], axis=1)

tags_expanded_cafe = df_cafe['태그리뷰'].apply(parse_tags).apply(pd.Series)
df_cafe = pd.concat([df_cafe.drop(columns=['태그리뷰']), tags_expanded_cafe], axis=1)

tags_expanded_drink = df_drink['태그리뷰'].apply(parse_tags).apply(pd.Series)
df_drink = pd.concat([df_drink.drop(columns=['태그리뷰']), tags_expanded_drink], axis=1)


In [25]:
df_meal.head()

,index,가게명,음식종류,음식이 맛있어요,가성비가 좋아요,혼밥하기 좋아요,친절해요,양이 많아요,재료가 신선해요,특별한 메뉴가 있어요,...,야외공간이 멋져요,컨셉이 독특해요,포장이 깔끔해요,술이 다양해요,직접 잘 구워줘요,커피가 맛있어요,디저트가 맛있어요,기본 안주가 좋아요,집중하기 좋아요,빵이 맛있어요
0,0,육쌈냉면 숙대점,냉면,419.0,165.0,125.0,64.0,64.0,44.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,장재근초밥,"초밥,롤",96.0,24.0,16.0,40.0,13.0,56.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,면식당 숙명여대점,일식당,373.0,126.0,188.0,128.0,151.0,153.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,포라임 숙대본점,베트남음식,869.0,307.0,419.0,328.0,276.0,327.0,36.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,베스트프렌드,종합분식,91.0,21.0,1.0,35.0,12.0,10.0,14.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
print(df_meal[df_meal['디저트가 맛있어요'].notna()])

    index 가게명 음식종류  음식이 맛있어요  가성비가 좋아요  혼밥하기 좋아요   친절해요  양이 많아요  재료가 신선해요  \
18     18   정  중식당     640.0     117.0      68.0  186.0    98.0     123.0   

    특별한 메뉴가 있어요  ...  야외공간이 멋져요  컨셉이 독특해요  포장이 깔끔해요  술이 다양해요  직접 잘 구워줘요  \
18        217.0  ...        NaN       1.0       NaN      NaN        NaN   

    커피가 맛있어요  디저트가 맛있어요  기본 안주가 좋아요  집중하기 좋아요  빵이 맛있어요  
18       1.0        1.0         NaN       NaN      NaN  

[1 rows x 52 columns]


In [27]:
print(df_meal[df_meal['빵이 맛있어요'].notna()])

    index        가게명  음식종류  음식이 맛있어요  가성비가 좋아요  혼밥하기 좋아요  친절해요  양이 많아요  \
55     55  메이드바이그레이스  샌드위치       NaN       NaN       NaN  72.0     NaN   

    재료가 신선해요  특별한 메뉴가 있어요  ...  야외공간이 멋져요  컨셉이 독특해요  포장이 깔끔해요  술이 다양해요  \
55       NaN         60.0  ...        NaN       NaN       NaN      NaN   

    직접 잘 구워줘요  커피가 맛있어요  디저트가 맛있어요  기본 안주가 좋아요  집중하기 좋아요  빵이 맛있어요  
55        NaN      35.0        NaN         NaN       NaN     81.0  

[1 rows x 52 columns]


In [28]:
# 모든 열의 타입을 int로 캐스팅 (숫자형 컬럼에 한하여)
df_meal = df_meal.apply(pd.to_numeric, errors='ignore')
df_cafe = df_cafe.apply(pd.to_numeric, errors='ignore')
df_drink = df_drink.apply(pd.to_numeric, errors='ignore')

<ipython-input-28-45fff6f98f87>:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_meal = df_meal.apply(pd.to_numeric, errors='ignore')
<ipython-input-28-45fff6f98f87>:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_cafe = df_cafe.apply(pd.to_numeric, errors='ignore')
<ipython-input-28-45fff6f98f87>:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_drink = df_drink.apply(pd.to_numeric, errors='ignore')


In [29]:
df_meal.fillna(0, inplace=True)
df_cafe.fillna(0, inplace=True)
df_drink.fillna(0, inplace=True)

In [33]:
# 행별 태그 총합 계산
df_meal['태그 총합'] = df_meal.select_dtypes(include=['number']).sum(axis=1)
df_cafe['태그 총합'] = df_cafe.select_dtypes(include=['number']).sum(axis=1)
df_drink['태그 총합'] = df_drink.select_dtypes(include=['number']).sum(axis=1)

# 각 태그의 비율 계산하여 새로운 데이터프레임 생성
df_meal_ratio = df_meal.select_dtypes(include=['number']).div(df_meal['태그 총합'], axis=0)
df_cafe_ratio = df_cafe.select_dtypes(include=['number']).div(df_cafe['태그 총합'], axis=0)
df_drink_ratio = df_drink.select_dtypes(include=['number']).div(df_drink['태그 총합'], axis=0)

# 결과 출력
print("df_cafe:\n", df_cafe)
print("\ndf_drink:\n", df_drink)
print("\ndf_meal:\n", df_meal)
print("\ndf_meal_ratio:\n", df_meal_ratio)
print("\ndf_cafe_ratio:\n", df_cafe_ratio)
print("\ndf_drink_ratio:\n", df_drink_ratio)

df_cafe:
      index           가게명    음식종류  빵이 맛있어요   친절해요  특별한 메뉴가 있어요  매장이 청결해요  \
11      11            앙꼬  카페,디저트      0.0    0.0          0.0       0.0   
12      12         루다브레드    베이커리    247.0  119.0         59.0      59.0   
13      13   컴포즈커피 숙명여대점      카페      0.0   88.0         37.0      46.0   
14      14   벤티프레소 숙명여대점      카페      1.0   10.0         14.0       9.0   
22      22         위드달걀빵    베이커리    153.0   69.0         86.0      60.0   
25      25     쥬케로카페 숙대점   브런치카페      1.0   35.0         73.0      25.0   
26      26       몬스터플레이스  카페,디저트      6.0  302.0        514.0     171.0   
27      27            킷테      카페      0.0   63.0         44.0      48.0   
29      29        카페 퓨엔테  카페,디저트      0.0   21.0         14.0      19.0   
30      30        카페나리나무  카페,디저트      7.0   88.0         37.0      39.0   
32      32      스타벅스 숙대점      카페      0.0  484.0         77.0     245.0   
34      34       스택빈 숙대점      카페      0.0   95.0         37.0     115.0   
43      43     

In [34]:
df_meal_ratio.head()

,index,음식이 맛있어요,가성비가 좋아요,혼밥하기 좋아요,친절해요,양이 많아요,재료가 신선해요,특별한 메뉴가 있어요,매장이 넓어요,매장이 청결해요,...,컨셉이 독특해요,포장이 깔끔해요,술이 다양해요,직접 잘 구워줘요,커피가 맛있어요,디저트가 맛있어요,기본 안주가 좋아요,집중하기 좋아요,빵이 맛있어요,태그 총합
0,0.000000,0.209500,0.082500,0.062500,0.032000,0.032000,0.022000,0.016000,0.014500,0.008500,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.001484,0.142433,0.035608,0.023739,0.059347,0.019288,0.083086,0.031157,0.004451,0.038576,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000580,0.108116,0.036522,0.054493,0.037101,0.043768,0.044348,0.019710,0.045507,0.037971,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.000458,0.132631,0.046856,0.063950,0.050061,0.042125,0.049908,0.005495,0.019536,0.027167,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.007782,0.177043,0.040856,0.001946,0.068093,0.023346,0.019455,0.027237,0.015564,0.011673,...,0.003891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
